In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
reads = pd.read_csv('../data/16p12_lcl_gene_reads_underscores.gct', sep='\t')
tpm   = pd.read_csv('../data/16p12_lcl_gene_tpm_underscores.gct', sep='\t')
mapp  = pd.read_csv('../data/gene_names_mapping_new.tsv', sep='\t')
dis   = pd.read_csv('../data/disease_genes_map.tsv', sep='\t')

rel = pd.read_csv('../data/family_summaries.tsv', sep='\t')
pheno = pd.read_csv('../data/pheno_final.tsv', sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
rel = rel.set_index('subject', drop=False)
pheno = pheno.set_index('sample', drop=False)

In [4]:
dis = dis.set_index('ensembl', drop=False)
mapp = mapp.set_index('ensembl', drop=False)

reads['Name'] = reads['Name'].apply(lambda s: s.split('.')[0])
reads = reads.set_index('Name')

tpm['Name'] = tpm['Name'].apply(lambda s: s.split('.')[0])
tpm = tpm.set_index('Name')
tpm = tpm.drop('Description', axis=1)

dis_dbs = ['Developmental_delay_DDD','DBD','DDDG2P','SFARI_Gene','Purcell_Schiz']

In [5]:
subjects = list(pheno.subject.unique())
subject_to_samples = {}
for subject in subjects:
    samples = pheno[pheno.subject == subject]['sample'].to_list()
    subject_to_samples[subject] = samples

def is_good_coverage_tpm(r):
    gene = r.name
    min_tpm = 0.2
    for subject in subjects:
        samples = subject_to_samples[subject]
        sub_tpms = r[samples]
#         print(sub_tpms)
        if (sub_tpms > .2).sum() == 3:
            return True
    return False

covered = tpm.T.apply(lambda r: is_good_coverage_tpm(r))

In [8]:
genes_covered = covered[covered].index.tolist()

In [10]:
len(genes_covered)

24340

In [11]:
tpm.shape

(56202, 96)

In [12]:
24340/56202

0.4330806732856482

In [13]:
with open('lcl_expressed.txt', 'w') as f:
    for s in genes_covered:
        f.write(s)
        f.write('\n')